# **This is 3rd Notebook for the Code**

# Utilizing BERT NLP Trained on Reviews

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid, RepeatedStratifiedKFold



from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Antoine's Path
# glassdoor_csv = pd.read_csv('/content/drive/MyDrive/Fall 2023/data 144/Data 144 Final Project/glassdoor_reviews.csv')

# Cleaned dataset
#glassdoor_csv = pd.read_csv('/content/drive/MyDrive/Data 144 Final Project/cleaned_glassdoor_reviews.csv')

# Sanjana's dataset
# glassdoor_csv = pd.read_csv('/content/drive/MyDrive/Data 144 Final Project/glassdoor_reviews.csv')

#Eric's Path
glassdoor_csv = pd.read_csv('/content/drive/MyDrive/Senior Year Classes/DATA 144: Data Mining/Data 144 Final Project/glassdoor_reviews.csv')
vader = pd.read_csv('/content/drive/MyDrive/Senior Year Classes/DATA 144: Data Mining/Data 144 Final Project/vader_lexicon.txt', sep = '\t', names= ['token', 'polarity'], index_col = 'token', usecols = ['token', 'polarity'])
company_stats_csv = pd.read_csv('/content/drive/MyDrive/Senior Year Classes/DATA 144: Data Mining/Data 144 Final Project/companies_sorted.csv')

# Part 1: Data Cleaning

In [ ]:
glassdoor_csv.replace({
   'x' : -1,
   'v' : 1,
   'r' : 0.5 ,
   'o' : 0
 },inplace = True)
glassdoor_csv

glassdoor_csv["culture_values"].fillna(np.mean(glassdoor_csv["culture_values"]), inplace = True)
glassdoor_csv["career_opp"].fillna(np.mean(glassdoor_csv["career_opp"]), inplace = True)
glassdoor_csv["comp_benefits"].fillna(np.mean(glassdoor_csv["comp_benefits"]), inplace = True)
glassdoor_csv["senior_mgmt"].fillna(np.mean(glassdoor_csv["senior_mgmt"]), inplace = True)
glassdoor_csv["work_life_balance"].fillna(np.mean(glassdoor_csv["work_life_balance"]), inplace = True)

null_dropped = glassdoor_csv.dropna(subset=['pros', 'cons', 'headline'])

for column in ["work_life_balance", "culture_values", "career_opp", "comp_benefits", "senior_mgmt"]:
  null_dropped[column] = null_dropped[column].astype(int)

null_dropped = null_dropped.dropna(subset=['diversity_inclusion'])

  # lowercase text
null_dropped['headline'] = null_dropped['headline'].str.lower()
null_dropped['pros'] = null_dropped['pros'].str.lower()
null_dropped['cons'] = null_dropped['cons'].str.lower()

<ipython-input-5-20cee15418ab>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_dropped[column] = null_dropped[column].astype(int)


# Part 2 Feature Engineering/Including Polarities

In [ ]:
glassdoor_csv = null_dropped

In [ ]:
glassdoor_csv.head()

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons
50,AFH-Wealth-Management,2020-10-01,Office Administrator,"Former Employee, more than 3 years","Bromsgrove, England, England",2,1,3,1.0,1,2,2,-1,0.0,-1.0,the people both make and destroy this place,"great people in some places, excellent christm...","poor pay, huge gap for pay between senior mana..."
51,AFH-Wealth-Management,2021-02-05,Quality Control,Former Employee,"Birmingham, England, England",1,3,1,2.0,1,1,1,-1,0.0,0.0,very low salaries,"majority of the people there are lovely, and t...",salaries are much lower than market competitor...
52,AFH-Wealth-Management,2021-02-07,IFA Administrator,"Former Employee, less than 1 year","Bromsgrove, England, England",4,3,3,4.0,4,4,2,1,0.0,1.0,good,"nice environment, love people, not too stressful",management can be clicky at times
53,AFH-Wealth-Management,2021-02-07,Investment Operations,"Former Employee, more than 1 year","Birmingham, England, England",3,5,5,4.0,3,1,2,-1,0.0,1.0,afh review,-great people\r\n-heading in a good direction ...,-low salary\r\n-middle management likes to mic...
55,AFH-Wealth-Management,2021-03-23,Administrative,Former Employee,NaN,1,5,1,2.0,1,2,1,-1,0.0,0.5,"great for people, not for work",you meet lovely people and make long life friends,it's not a good work environment and you're no...


In [ ]:
glassdoor_csv.columns

Index(['firm', 'date_review', 'job_title', 'current', 'location',
       'overall_rating', 'work_life_balance', 'culture_values',
       'diversity_inclusion', 'career_opp', 'comp_benefits', 'senior_mgmt',
       'recommend', 'ceo_approv', 'outlook', 'headline', 'pros', 'cons'],
      dtype='object')

In [ ]:
def sanitize_texts(df):
  punct_re = r'[^\s\w]'
  df["headline_clean"] = df["headline"].apply(lambda string: str(string).lower()).str.replace(punct_re, " ")
  df["pros_clean"] = df["pros"].apply(lambda string: str(string).lower()).str.replace(punct_re, " ")
  df["cons_clean"] = df["cons"].apply(lambda string: str(string).lower()).str.replace(punct_re, " ")
  return df

sanitized_glassdoor = sanitize_texts(glassdoor_csv)

def to_tidy_format(df, column): # Column is the string
    tidy = (
        df[column + "_clean"]
        .str.split()
        .explode()
        .to_frame()
        .rename(columns={column + "_clean": "word"})
    )
    return tidy

pros_words = to_tidy_format(sanitized_glassdoor, "pros").reset_index()
cons_words = to_tidy_format(sanitized_glassdoor, "cons").reset_index()
headline_words = to_tidy_format(sanitized_glassdoor, "headline").reset_index()

def add_polarity(df, tidy_df, column):
    df[column + "_polarity"] = (
        tidy_df
        .merge(vader, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index', 'index')
        .sum()
        .fillna(0)
    )
    return df

def vader_clean_and_compute(df):
  # Cleaning the text for preprocessing
  add_polarity(df, pros_words, "pros")
  add_polarity(df, cons_words, "cons")
  add_polarity(df, headline_words, "headline")
  return df

def standardize_polarity(df):
  #df['pros_polarity'] = (df['pros_polarity'] - df['pros_polarity'].mean()) / df['pros_polarity'].std()
  df['pros_polarity'] = np.log(df['pros_polarity'] + 0.1)
  #df['cons_polarity'] = (df['cons_polarity'] - df['cons_polarity'].mean()) / df['cons_polarity'].std()
  df['cons_polarity'] = np.log(np.abs(df['cons_polarity']) - 0.1)
  #df['headline_polarity'] = (df['headline_polarity'] - df['headline_polarity'].mean()) / df['headline_polarity'].std()
  df['headline_polarity'] = np.log(np.abs(df['headline_polarity']) + 0.1)
  return df

df_preprocessed = standardize_polarity(vader_clean_and_compute(glassdoor_csv)).replace([np.inf, -np.inf], np.nan).dropna()
df_preprocessed.head()

<ipython-input-9-0277e3c27fbb>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["headline_clean"] = df["headline"].apply(lambda string: str(string).lower()).str.replace(punct_re, " ")
<ipython-input-9-0277e3c27fbb>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["pros_clean"] = df["pros"].apply(lambda string: str(string).lower()).str.replace(punct_re, " ")
<ipython-input-9-0277e3c27fbb>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["cons_clean"] = df["cons"].apply(lambda string: str(string).lower()).str.replace(punct_re, " ")
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = ge

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,...,outlook,headline,pros,cons,headline_clean,pros_clean,cons_clean,pros_polarity,cons_polarity,headline_polarity
50,AFH-Wealth-Management,2020-10-01,Office Administrator,"Former Employee, more than 3 years","Bromsgrove, England, England",2,1,3,1.0,1,...,-1.0,the people both make and destroy this place,"great people in some places, excellent christm...","poor pay, huge gap for pay between senior mana...",the people both make and destroy this place,great people in some places excellent christm...,poor pay huge gap for pay between senior mana...,2.028148,0.641854,0.955511
51,AFH-Wealth-Management,2021-02-05,Quality Control,Former Employee,"Birmingham, England, England",1,3,1,2.0,1,...,0.0,very low salaries,"majority of the people there are lovely, and t...",salaries are much lower than market competitor...,very low salaries,majority of the people there are lovely and t...,salaries are much lower than market competitor...,1.064711,1.481605,0.182322
53,AFH-Wealth-Management,2021-02-07,Investment Operations,"Former Employee, more than 1 year","Birmingham, England, England",3,5,5,4.0,3,...,1.0,afh review,-great people\r\n-heading in a good direction ...,-low salary\r\n-middle management likes to mic...,afh review,great people\r\n heading in a good direction ...,low salary\r\n middle management likes to mic...,1.629241,-1.203973,-2.302585
60,AFH-Wealth-Management,2021-05-12,Client Engagement Executive,"Former Employee, less than 1 year","Birmingham, England, England",1,1,2,1.0,1,...,-1.0,terrible- avoid like the plague,"none, they lie about working hours, bonuses, p...",-unachievable bonus unfairly set by manager\n-...,terrible avoid like the plague,none they lie about working hours bonuses p...,unachievable bonus unfairly set by manager\n ...,0.832909,2.054124,0.641854
61,AFH-Wealth-Management,2021-05-13,Administrative Assistant,"Former Employee, more than 1 year","Worcester, England, England",1,1,1,1.0,1,...,0.5,steer clear,there are none to speak of,"borderline bullying, nepotism, overworked - st...",steer clear,there are none to speak of,borderline bullying nepotism overworked st...,-2.302585,0.182322,0.530628


#Part 3: NLP
We will be using the [bert-base-multilingual-uncased model](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) finetuned for sentiment analysis to get a prediction (1-5) based on the review

In [ ]:
pip install -q transformers

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
df_preprocessed['headline + pros + cons'] = df_preprocessed['headline_clean'] + ' ' + df_preprocessed['pros_clean'] + ' ' + df_preprocessed['cons_clean']
df_preprocessed.head()

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,...,headline,pros,cons,headline_clean,pros_clean,cons_clean,pros_polarity,cons_polarity,headline_polarity,headline + pros + cons
50,AFH-Wealth-Management,2020-10-01,Office Administrator,"Former Employee, more than 3 years","Bromsgrove, England, England",2,1,3,1.0,1,...,the people both make and destroy this place,"great people in some places, excellent christm...","poor pay, huge gap for pay between senior mana...",the people both make and destroy this place,great people in some places excellent christm...,poor pay huge gap for pay between senior mana...,2.028148,0.641854,0.955511,the people both make and destroy this place gr...
51,AFH-Wealth-Management,2021-02-05,Quality Control,Former Employee,"Birmingham, England, England",1,3,1,2.0,1,...,very low salaries,"majority of the people there are lovely, and t...",salaries are much lower than market competitor...,very low salaries,majority of the people there are lovely and t...,salaries are much lower than market competitor...,1.064711,1.481605,0.182322,very low salaries majority of the people there...
53,AFH-Wealth-Management,2021-02-07,Investment Operations,"Former Employee, more than 1 year","Birmingham, England, England",3,5,5,4.0,3,...,afh review,-great people\r\n-heading in a good direction ...,-low salary\r\n-middle management likes to mic...,afh review,great people\r\n heading in a good direction ...,low salary\r\n middle management likes to mic...,1.629241,-1.203973,-2.302585,afh review great people\r\n heading in a good...
60,AFH-Wealth-Management,2021-05-12,Client Engagement Executive,"Former Employee, less than 1 year","Birmingham, England, England",1,1,2,1.0,1,...,terrible- avoid like the plague,"none, they lie about working hours, bonuses, p...",-unachievable bonus unfairly set by manager\n-...,terrible avoid like the plague,none they lie about working hours bonuses p...,unachievable bonus unfairly set by manager\n ...,0.832909,2.054124,0.641854,terrible avoid like the plague none they lie...
61,AFH-Wealth-Management,2021-05-13,Administrative Assistant,"Former Employee, more than 1 year","Worcester, England, England",1,1,1,1.0,1,...,steer clear,there are none to speak of,"borderline bullying, nepotism, overworked - st...",steer clear,there are none to speak of,borderline bullying nepotism overworked st...,-2.302585,0.182322,0.530628,steer clear there are none to speak of borderl...


In [ ]:
def bert_rating(df):
  df['nlp rating'] = df['headline + pros + cons'].apply(lambda x: int(sentiment_pipeline(str(x)[0:512])[0]['label'][0]))
  #df['nlp rating confidence'] = df['headline + pros + cons'].apply(lambda x: sentiment_pipeline(str(x)[0:512])[0]['score'][0])
  return df

### Running on 5% of the data (~4000 rows), as the bert NLP model takes considerable amount of time.

In [ ]:
sample = df_preprocessed.sample(frac=0.05, random_state=42)
with_nlp = bert_rating(sample)
with_nlp.head()

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,...,pros,cons,headline_clean,pros_clean,cons_clean,pros_polarity,cons_polarity,headline_polarity,headline + pros + cons,nlp rating
92685,Barclays,2020-12-15,CRM Manager,"Current Employee, more than 5 years","London, England, England",4,4,4,4.0,3,...,multiple career paths available to pursue,limited promotion opportunities as you get hig...,decent place to work,multiple career paths available to pursue,limited promotion opportunities as you get hig...,-2.302585,-0.510826,-2.302585,decent place to work multiple career paths ava...,3
213937,Deloitte,2021-03-14,Consultant,"Former Employee, more than 1 year","New York, NY",2,1,3,1.0,4,...,"good tenure of couple of years, did some great...",poor immigration support - their new immigrati...,not ideal for employees on visa,good tenure of couple of years did some great...,poor immigration support their new immigrati...,2.251292,1.504077,0.916291,not ideal for employees on visa good tenure of...,2
711175,Roche,2021-03-31,Statistical Programmer Analyst,"Current Employee, less than 1 year","Welwyn, England, England",5,5,5,4.0,5,...,"very good benefits scheme, generous bonus, lar...",nothing significant - would benefit from a bit...,great people and work life balance,very good benefits scheme generous bonus lar...,nothing significant would benefit from a bit...,2.484907,0.993252,1.163151,great people and work life balance very good b...,4
741240,Salesforce,2021-04-15,Salesforce Support Engineer,Former Employee,Hyderābād,4,3,5,5.0,4,...,employee benefits are much better,have to meet the metrics as a part of customer...,none,employee benefits are much better,have to meet the metrics as a part of customer...,1.280934,0.587787,-2.302585,none employee benefits are much better have to...,2
501116,Marriott-International,2021-05-31,Assistant Manager,"Current Employee, more than 3 years",Mumbai,5,5,5,5.0,5,...,awesome company with good work environment,best place to work with good employee engagement,best place to work,awesome company with good work environment,best place to work with good employee engagement,1.629241,1.945910,1.193922,best place to work awesome company with good w...,5


In [ ]:
with_nlp.shape

(3759, 26)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(list(with_nlp['overall_rating']), list(with_nlp['nlp rating']))
accuracy

0.4644852354349561

# Part 4: Sorting by NLP Rating on the review, and seeing what companies show up
#### We are limiting this to companies with >20 reviews in this sample

In [ ]:
grouped = with_nlp.groupby('firm').agg(['mean', 'size'])
filtered_more_15 = grouped[grouped['overall_rating']['size'] > 25]
sorted = filtered_more_15.sort_values(by=('nlp rating', 'mean'), ascending = False)
sorted

<ipython-input-61-497999338e88>:1: FutureWarning: ['date_review', 'job_title', 'current', 'location', 'headline', 'pros', 'cons', 'headline_clean', 'pros_clean', 'cons_clean', 'headline + pros + cons'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped = with_nlp.groupby('firm').agg(['mean', 'size'])


overall_rating      work_life_balance       \
                                 mean size              mean size   
firm                                                                
Salesforce                   4.333333   48          4.020833   48   
American-Express             4.346154   26          3.846154   26   
Google                       4.383333   60          4.083333   60   
Marriott-International       4.340426   47          3.914894   47   
CBRE                         3.758621   29          3.068966   29   
Microsoft                    4.069767   86          3.790698   86   
Apple                        4.121622   74          3.513514   74   
SAP                          4.395349   43          4.488372   43   
IBM                          4.013393  224          4.022321  224   
Deloitte                     3.745174  259          2.888031  259   
EY                           3.662577  163          2.631902  163   
Morgan-Stanley               3.629630   27          2.703704   27   
PwC                          3.647799  159          2.685535  159   
Oracle                       3.666667  126          3.873016  126   
KPMG                         3.575540  139          2.733813  139   
J-P-Morgan                   3.862385  109          3.440367  109   
Pizza-Hut                    3.457143   35          3.085714   35   
Barclays                     3.700000   40          3.800000   40   
NHS                          4.060606   33          3.606061   33   
Tesco                        3.553846   65          3.200000   65   
Citi                         3.800000   70          3.314286   70   
Vodafone                     3.750000   36          3.638889   36   
HSBC-Holdings                3.788462   52          3.826923   52   
BT                           3.407407   27          3.629630   27   
McDonald-s                   3.352459  244          3.241803  244   
Goldman-Sachs                3.743590   39          2.743590   39   
GlaxoSmithKline              3.392857   28          3.428571   28   
BNY-Mellon                   3.187500   32          3.406250   32   

                       culture_values      diversity_inclusion       \
                                 mean size                mean size   
firm                                                                  
Salesforce                   4.395833   48            4.333333   48   
American-Express             4.423077   26            4.538462   26   
Google                       4.200000   60            4.166667   60   
Marriott-International       4.319149   47            4.468085   47   
CBRE                         3.724138   29            3.655172   29   
Microsoft                    4.186047   86            4.139535   86   
Apple                        4.162162   74            4.445946   74   
SAP                          4.488372   43            4.627907   43   
IBM                          4.080357  224            4.308036  224   
Deloitte                     3.776062  259            3.949807  259   
EY                           3.693252  163            4.116564  163   
Morgan-Stanley               3.777778   27            4.111111   27   
PwC                          3.660377  159            3.911950  159   
Oracle                       3.531746  126            3.809524  126   
KPMG                         3.611511  139            3.755396  139   
J-P-Morgan                   3.880734  109            4.146789  109   
Pizza-Hut                    3.171429   35            3.457143   35   
Barclays                     3.875000   40            3.825000   40   
NHS                          3.878788   33            4.060606   33   
Tesco                        3.246154   65            3.784615   65   
Citi                         3.457143   70            3.857143   70   
Vodafone                     3.611111   36            3.944444   36   
HSBC-Holdings                3.596154   52            3.865385   52   
BT                           3.407407

In [ ]:
sorted[['overall_rating', 'nlp rating']]

overall_rating      nlp rating     
                                 mean size       mean size
firm                                                      
Salesforce                   4.333333   48   4.208333   48
American-Express             4.346154   26   4.115385   26
Google                       4.383333   60   4.083333   60
Marriott-International       4.340426   47   4.063830   47
CBRE                         3.758621   29   4.034483   29
Microsoft                    4.069767   86   3.988372   86
Apple                        4.121622   74   3.972973   74
SAP                          4.395349   43   3.860465   43
IBM                          4.013393  224   3.799107  224
Deloitte                     3.745174  259   3.675676  259
EY                           3.662577  163   3.668712  163
Morgan-Stanley               3.629630   27   3.666667   27
PwC                          3.647799  159   3.654088  159
Oracle                       3.666667  126   3.634921  126
KPMG                         3.575540  139   3.633094  139
J-P-Morgan                   3.862385  109   3.623853  109
Pizza-Hut                    3.457143   35   3.600000   35
Barclays                     3.700000   40   3.600000   40
NHS                          4.060606   33   3.575758   33
Tesco                        3.553846   65   3.553846   65
Citi                         3.800000   70   3.514286   70
Vodafone                     3.750000   36   3.500000   36
HSBC-Holdings                3.788462   52   3.461538   52
BT                           3.407407   27   3.407407   27
McDonald-s                   3.352459  244   3.401639  244
Goldman-Sachs                3.743590   39   3.384615   39
GlaxoSmithKline              3.392857   28   3.214286   28
BNY-Mellon                   3.187500   32   3.093750   32